<a href="https://colab.research.google.com/github/acmilannesta/Bert-embedding/blob/master/GoogleQA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Tue Dec 10 14:44:16 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.36       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    35W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
!pip install keras-bert
!wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
!unzip uncased_L-12_H-768_A-12.zip

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
import numpy as np
import pandas as pd
import os, gc
import codecs
from keras.layers import *
from keras.callbacks import Callback
from keras.models import Model, load_model
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import StratifiedKFold, RepeatedStratifiedKFold, GroupKFold, KFold
from scipy.stats import spearmanr
from tqdm import tqdm
from keras_bert import load_trained_model_from_checkpoint, Tokenizer, AdamWarmup, calc_train_steps, get_custom_objects
from hyperopt import fmin, hp, tpe, STATUS_OK, Trials


Using TensorFlow backend.


In [0]:
data_path = 'drive/My Drive/GoogleQA/Data/'
train = pd.read_csv(data_path+'train.csv')
test = pd.read_csv(data_path+'test.csv')
sub = pd.read_csv(data_path+'sample_submission.csv')

train_q = pd.read_csv(data_path+'train_q.csv')
test_q = pd.read_csv(data_path+'test_q.csv')
train_a = pd.read_csv(data_path+'train_answer.csv')
test_a = pd.read_csv(data_path+'test_answer.csv')

# df_aux = pd.concat([train[['host', 'category']], test[['host', 'category']]], 0)
# aux1 = pd.get_dummies(df_aux['category'], drop_first=True)
# aux2 = pd.get_dummies(df_aux['host'].apply(lambda x: x.split('.')[0]), drop_first=True)
# aux = pd.concat([aux1, aux2], 1)
# train_aux = pd.concat([train, aux.iloc[:len(train), ]], 1)
# test_aux =  pd.concat([test, aux.iloc[len(train):, ]], 1)

In [0]:
test.columns

Index(['qa_id', 'question_title', 'question_body', 'question_user_name',
       'question_user_page', 'answer', 'answer_user_name', 'answer_user_page',
       'url', 'category', 'host'],
      dtype='object')

##Chunknize answer

In [0]:
# def answer_chunk(df):
#     answer_col = ['qa_id'] + [col for col in df.columns if col.startswith('answer') and
#                 col not in ['answer_user_name', 'answer_user_page']] 
#     answer = df[answer_col]

#     answer_chunk = pd.DataFrame()
#     for row in tqdm(range(len(answer))):
#         if len(answer.loc[row, 'answer']) % 256 != 0:
#             replicate_row = len(answer.loc[row, 'answer']) // 256 + 1
#         else:
#             replicate_row = len(answer.loc[row, 'answer']) // 256
#         for i in range(replicate_row):
#             tmp = answer.loc[row].copy()
#             tmp.loc['answer_chunk'] = tmp['answer'][i*256: (i+1)*256]
#             answer_chunk = answer_chunk.append(tmp, ignore_index=True)
#     return answer_chunk

# def question_chunk(df):
#     q_col = ['qa_id'] + [col for col in df.columns if col.startswith('question') and
#                 col not in ['question_user_name', 'question_user_name']] 
#     q = df[q_col]

#     q_chunk = pd.DataFrame()
#     for row in tqdm(range(len(q))):
#         if len(q.loc[row, 'question_body']) % 256 != 0:
#             replicate_row = len(q.loc[row, 'question_body']) // 256 + 1
#         else:
#             replicate_row = len(q.loc[row, 'question_body']) // 256
#         for i in range(replicate_row):
#             tmp = q.loc[row].copy()
#             tmp.loc['qbody_chunk'] = tmp['question_body'][i*256: (i+1)*256]
#             q_chunk = q_chunk.append(tmp, ignore_index=True)
#     return q_chunk

# train_q, test_q = question_chunk(train), question_chunk(test)
# train_a, test_a = answer_chunk(train), answer_chunk(test)

100%|██████████| 476/476 [00:09<00:00, 52.14it/s]


In [0]:
MAXLEN = 256 #@param {type:"slider", min:128, max:512, step:32}
BATCH_SIZE = 8 #@param {type:'slider', min:8, max:32, step:8}
NUM_EPOCHS = 3
NUM_CLASSES = 30
LR = 5e-5
MIN_LR = 0
# OUTPUT_TRAIN = 'train_bert_ipredcv1415_oof.csv'
# OUTPUT_TEST = 'test_bert_large.npy'
model_path = 'uncased_L-12_H-768_A-12' #@param ['uncased_L-12_H-768_A-12', 'wwm_uncased_L-24_H-1024_A-16', 'uncased_L-24_H-1024_A-16']
target_q_col = train.columns.tolist()[11:32]
target_a_col = train.columns.tolist()[32:42]

In [0]:
token_dict = {}
with codecs.open(os.path.join(model_path, 'vocab.txt'), 'r', 'utf8') as reader:
    for line in reader:
        token = line.strip()
        token_dict[token] = len(token_dict)
tokenizer = Tokenizer(token_dict)

In [0]:
def convert_data(data_df, branch='training', type='answer'):
    data_df = data_df.reset_index(drop=True)
    global tokenizer
    global MAXLEN
    global target_a_col
    global target_q_col
    q_title, q_body, answer = [], [], []
    if type=='answer':
        for i in tqdm(range(len(data_df))):
            answer_ids, _ = tokenizer.encode(data_df.loc[i, 'answer_chunk'][:MAXLEN])
            answer.append(answer_ids)
        if branch == 'training':
            targets = data_df[target_a_col]
            return answer, np.array(targets)
        else:
            return answer
    else:     
        for i in tqdm(range(len(data_df))):                  
            q_title_ids, _ = tokenizer.encode(data_df.loc[i, 'question_title'][:MAXLEN])
            q_title.append(q_title_ids)
            q_body_ids, _ = tokenizer.encode(data_df.loc[i, 'question_body'][:MAXLEN])
            q_body.append(q_body_ids)
        if branch == 'training':
            targets = data_df[target_q_col]
            return [q_title, q_body], np.array(targets)
        else:
            return [q_title, q_body]



In [0]:
"""## Data Generator"""
def seq_padding(X, padding=0):
    L = [len(x) for x in X]
    ML = max(L)
    return np.array([np.concatenate([x, [padding] * (ML - len(x))]) if len(x) < ML else x for x in X])

class data_generator:
    def __init__(self, data, batch_size=BATCH_SIZE, branch='train', q_a='a'):
        self.data = data
        self.batch_size = batch_size
        self.branch = branch
        self.q_a = q_a
        self.steps = len(self.data) // self.batch_size
        if len(self.data) % self.batch_size != 0:
            self.steps += 1

    def __len__(self):
        return self.steps

    def __iter__(self):
        while True:
            if self.branch == 'train':
                np.random.shuffle(self.data)
            for i in range(self.steps):
                d = self.data[i * self.batch_size: (i + 1) * self.batch_size]
                if self.q_a == 'q':
                    X1 = seq_padding([x[0] for x in d])
                    X2 = np.zeros_like(X1)
                    X3 = seq_padding([x[1] for x in d])
                    X4 = np.zeros_like(X3)
                    if self.branch == 'test':
                        # aux = np.array([x[3] for x in d])
                        yield [X1, X2, X3, X4]
                    else:
                        Y = np.array([x[2] for x in d])
                        # aux = np.array([x[4] for x in d])
                        yield [X1, X2, X3, X4], Y
                else:
                    X1 = seq_padding([x[0] for x in d])
                    X2 = np.zeros_like(X1)
                    if self.branch == 'test':
                        # aux = np.array([x[3] for x in d])
                        yield [X1, X2]
                    else:
                        Y = np.array([x[1] for x in d])
                        # aux = np.array([x[4] for x in d])
                        yield [X1, X2], Y
        


In [0]:
def model_build(len_train, q_a='a'):
    global NUM_CLASSES
    global BATCH_SIZE
    global NUM_EPOCHS
    global MIN_LR
    global LR
    global MAXLEN

    bert_model = load_trained_model_from_checkpoint(
        os.path.join(model_path, 'bert_config.json'),
        os.path.join(model_path, 'bert_model.ckpt'),
        seq_len = MAXLEN,
        trainable=True,
    )

    if q_a == 'q':
        q_in = Input(shape=(None,))
        q2_in = Input(shape=(None,))
        qb_in = Input(shape=(None,))
        qb2_in = Input(shape=(None,))

        q_inputs = bert_model([q_in, q2_in])
        q_outputs = Lambda(lambda x: x[:, 0])(q_inputs)
        qb_inputs = bert_model([qb_in, qb2_in])
        qb_outputs = Lambda(lambda x: x[:, 0])(qb_inputs)

        dense = concatenate([q_outputs, qb_outputs])
        outputs = Dense(len(target_q_col), activation='softmax')(dense)
        model = Model([q_in, q2_in, qb_in, qb2_in], outputs)
    else:
        a_in = Input(shape=(None,))
        a2_in = Input(shape=(None,))
        # aux_in = Input(shape=(aux.shape[1], ))

        a_inputs = bert_model([a_in, a2_in])
        a_outputs = Lambda(lambda x: x[:, 0])(a_inputs)

        dense = a_outputs
        outputs = Dense(len(target_a_col), activation='softmax')(dense)
        model = Model([a_in, a2_in], outputs)

    decay_steps, warmup_steps = calc_train_steps(
        len_train,
        batch_size=BATCH_SIZE,
        epochs=NUM_EPOCHS,
    )

    model.compile(
        loss='binary_crossentropy',
        # optimizer=Adam(1e-5),
        optimizer=AdamWarmup(
            decay_steps=decay_steps,
            warmup_steps=warmup_steps,
            lr=LR,
            min_lr=MIN_LR,
            ))
    del bert_model
    gc.collect()
    return model


## train on answer

In [0]:
test_x = convert_data(test_a, branch='testing')
pred = np.zeros((len(test_a), NUM_CLASSES))
kf = KFold(n_splits=5, shuffle=True, random_state=0)
idx = [x for x in kf.split(train_a)]

def compute_spearmanr(trues, preds):
    rhos = []
    for col_trues, col_pred in zip(trues.T, preds.T):
        rhos.append(spearmanr(col_trues, col_pred + np.random.normal(0, 1e-7, col_pred.shape[0])).correlation)
    return np.nanmean(rhos)

class IntervalEval(Callback):
    def __init__(self, test_data, label):
        super(Callback, self).__init__()
        self.test_data = test_data
        self.label = label

    def on_epoch_end(self, epoch, logs={}):
        # if epoch == 3:
        preds = self.model.predict_generator(self.test_data.__iter__(), len(self.test_data), verbose=1)
        score = compute_spearmanr(self.label, preds)
        print('Spearman - {:.5f}'.format(score))

for i, (tr_idx, val_idx) in enumerate(idx[0:1], 1):
    print('\nFold - {:}\n'.format(i))
    tr, val = train_a.loc[tr_idx], train_a.loc[val_idx]
    tr_x, tr_y = convert_data(tr)
    val_x, val_y = convert_data(val)
    
    model = model_build(len_train=len(tr))


    train_D = data_generator(list(zip(tr_x, tr_y)))
    valid_D = data_generator(list(zip(val_x, val_y)), branch='valid')
    test_D = data_generator(list(zip(test_x)), branch='test')
    ieval = IntervalEval(test_data=valid_D, label=val_y)
    model.fit_generator(
        train_D.__iter__(),
        steps_per_epoch=len(train_D),
        epochs=NUM_EPOCHS,
        callbacks = [ieval]
    )
    # oof_pred = model.predict_generator(valid_D.__iter__(), len(valid_D), verbose=1)
    # train_aug.loc[val_idx, 'oof_pred'] = np.argmax(oof_pred, 1)
    # print('oof - {:} f1_score - {:.4f}'.format(i, spearmanr(val_y, np.argmax(oof_pred, 1), average='weighted')))

    pred+= model.predict_generator(test_D.__iter__(), len(test_D), verbose=1) / kf.get_n_splits()
    # np.save(OUTPUT_TEST, pred)
    # s3.upload_file(Filename=OUTPUT_TEST, Bucket='acmilannesta', Key='large/'+OUTPUT_TEST)

    # model_file = 'model-oof-'+str(i)+'.h5'
    # model.save('model.h5')

    del model
    gc.collect()



Epoch 1/3
  80/2291 [>.............................] - ETA: 11:31 - loss: 1.6316

In [0]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
model_2 (Model)                 multiple             108695040   input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 768)          0           model_2[1][0]              